In [94]:
# !pip --trusted-host pypi.org --trusted-host files.pythonhosted.org install xlrd

In [84]:
import re
import os
import glob
import numpy as np
import pandas as pd
from datetime import datetime

from selenium import webdriver
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [223]:
DOWNLOAD_PATH = rf'C:\Users\{os.environ["USERNAME"]}\Downloads'
TENOR_MAP = {'3월': 0.25, '6월': 0.5, '9월': 0.75, '1년': 1.0, '1년6월': 1.5, '2년': 2., '2년6월': 2.5, '3년': 3., '4년': 4., '5년': 5., '7년': 7., '10년': 10., '15년': 15., '20년': 20., '30년': 30., '50년': 50.}

In [233]:
# fx_tenor, fx_setting
FILENAME = "data/FSS_IFRS17 및 K-ICS 할인율 산출기준(원화, 해외통화)_'21년.xlsx"
df = pd.read_excel(FILENAME, sheet_name='IFRS17 및 K-ICS 할인율 산출기준')

currencies = df.iloc[8, 2:].values
num_currencies = len(currencies)
llp = df.iloc[10, 2:2+num_currencies].values.astype(float)
cp = df.iloc[11, 2:2+num_currencies].values.astype(float)
ltfr = df.iloc[12, 2:2+num_currencies].values.astype(float)
va = df.iloc[13, 2:2+num_currencies].values.astype(float)
lp = df.iloc[14, 2:2+num_currencies].values.astype(float)
freq = df.iloc[15, 2:2+num_currencies].values.astype(int)

fx_setting = pd.DataFrame(np.c_[currencies, llp, cp, ltfr,va, lp, freq],
    columns=['CURRENCY', 'LLP', 'CP', 'LTFR', 'VA', 'LP', 'FREQ'])

llp = float(re.search(r'(\d+)년', df.iloc[2, 1]).group(1))
cp = float(re.search(r': ([0-9]+)년', df.iloc[3, 1]).group(1))
ltfr = float(re.search(r': ([0-9\.]+)', df.iloc[4, 1]).group(1))/100

tenor_end = df.iloc[17:, 1].str.find('※ 참고사항').dropna().index[0]
tenor_all = df.iloc[17:tenor_end-1, 1].values.astype(float)
num_tenor = len(tenor_all)

fx_tickers = pd.DataFrame(df.iloc[17:17+num_tenor,2:2+num_currencies].values, columns = currencies, index=tenor).T
result = []
for curr, tick in fx_tickers.iterrows():
    result.append([curr, tick.loc[lambda x: x != '-'].index.to_numpy()])
fx_tenor = pd.DataFrame(result,
    columns=['CURRENCY', 'TENOR'])

In [2]:
driver = webdriver.Chrome()

In [19]:
driver.get('https://kofiabond.or.kr/websquare/websquare.html?w2xPath=/xml/Com/Common_TabMnuDsp.xml&divisionId=MBIS01070010000000&serviceId=BIS0100100280&topMenuIndex=6&w2xHome=/xml/&w2xDocumentRoot=#tabContents1_contents_tabs2_bridge')

In [21]:
driver.find_element(By.CSS_SELECTOR, '#tabContents1_tab_tabs2').click()

In [22]:
driver.switch_to.frame(driver.find_element(By.CSS_SELECTOR, '#tabContents1_contents_tabs2_body'))

In [62]:
start_date = '20210101'
end_date = '20211231'

driver.find_element(By.CSS_SELECTOR, '#schSstandardDt_input').click()
driver.find_element(By.CSS_SELECTOR, '#schSstandardDt_input').send_keys(Keys.DELETE)
driver.find_element(By.CSS_SELECTOR, '#schSstandardDt_input').send_keys(Keys.DELETE)
driver.find_element(By.CSS_SELECTOR, '#schSstandardDt_input').send_keys(Keys.DELETE)
driver.find_element(By.CSS_SELECTOR, '#schSstandardDt_input').send_keys(Keys.DELETE)
driver.find_element(By.CSS_SELECTOR, '#schSstandardDt_input').send_keys(Keys.BACKSPACE)
driver.find_element(By.CSS_SELECTOR, '#schSstandardDt_input').send_keys(Keys.BACKSPACE)
driver.find_element(By.CSS_SELECTOR, '#schSstandardDt_input').send_keys(Keys.BACKSPACE)
driver.find_element(By.CSS_SELECTOR, '#schSstandardDt_input').send_keys(Keys.BACKSPACE)
driver.find_element(By.CSS_SELECTOR, '#schSstandardDt_input').send_keys(start_date)

driver.find_element(By.CSS_SELECTOR, '#schEstandardDt_input').click()
driver.find_element(By.CSS_SELECTOR, '#schEstandardDt_input').send_keys(Keys.DELETE)
driver.find_element(By.CSS_SELECTOR, '#schEstandardDt_input').send_keys(Keys.DELETE)
driver.find_element(By.CSS_SELECTOR, '#schEstandardDt_input').send_keys(Keys.DELETE)
driver.find_element(By.CSS_SELECTOR, '#schEstandardDt_input').send_keys(Keys.DELETE)
driver.find_element(By.CSS_SELECTOR, '#schEstandardDt_input').send_keys(Keys.BACKSPACE)
driver.find_element(By.CSS_SELECTOR, '#schEstandardDt_input').send_keys(Keys.BACKSPACE)
driver.find_element(By.CSS_SELECTOR, '#schEstandardDt_input').send_keys(Keys.BACKSPACE)
driver.find_element(By.CSS_SELECTOR, '#schEstandardDt_input').send_keys(Keys.BACKSPACE)
driver.find_element(By.CSS_SELECTOR, '#schEstandardDt_input').send_keys(end_date)

Select(driver.find_element(By.CSS_SELECTOR, '#selectBndTyp1_input_0')).select_by_index(1)
Select(driver.find_element(By.CSS_SELECTOR, '#selectBndTyp2_input_0')).select_by_index(1)
Select(driver.find_element(By.CSS_SELECTOR, '#selectBndTyp3_input_0')).select_by_index(1)
Select(driver.find_element(By.CSS_SELECTOR, '#selectBndTyp4_input_0')).select_by_index(1)
Select(driver.find_element(By.CSS_SELECTOR, '#selectbox_input_0')).select_by_index(2)

Select(driver.find_element(By.CSS_SELECTOR, '#selectTrm1_input_0')).select_by_index(1)
Select(driver.find_element(By.CSS_SELECTOR, '#selectTrm2_input_0')).select_by_index(2)
Select(driver.find_element(By.CSS_SELECTOR, '#selectTrm3_input_0')).select_by_index(3)
Select(driver.find_element(By.CSS_SELECTOR, '#selectTrm4_input_0')).select_by_index(4)


In [63]:
driver.execute_script('searchData()')

In [ ]:
driver.execute_script('excelDownLoad()')

In [67]:
start_date = '20210101'
end_date = '20211231'

driver.find_element(By.CSS_SELECTOR, '#schSstandardDt_input').click()
driver.find_element(By.CSS_SELECTOR, '#schSstandardDt_input').send_keys(Keys.DELETE)
driver.find_element(By.CSS_SELECTOR, '#schSstandardDt_input').send_keys(Keys.DELETE)
driver.find_element(By.CSS_SELECTOR, '#schSstandardDt_input').send_keys(Keys.DELETE)
driver.find_element(By.CSS_SELECTOR, '#schSstandardDt_input').send_keys(Keys.DELETE)
driver.find_element(By.CSS_SELECTOR, '#schSstandardDt_input').send_keys(Keys.BACKSPACE)
driver.find_element(By.CSS_SELECTOR, '#schSstandardDt_input').send_keys(Keys.BACKSPACE)
driver.find_element(By.CSS_SELECTOR, '#schSstandardDt_input').send_keys(Keys.BACKSPACE)
driver.find_element(By.CSS_SELECTOR, '#schSstandardDt_input').send_keys(Keys.BACKSPACE)
driver.find_element(By.CSS_SELECTOR, '#schSstandardDt_input').send_keys(start_date)

driver.find_element(By.CSS_SELECTOR, '#schEstandardDt_input').click()
driver.find_element(By.CSS_SELECTOR, '#schEstandardDt_input').send_keys(Keys.DELETE)
driver.find_element(By.CSS_SELECTOR, '#schEstandardDt_input').send_keys(Keys.DELETE)
driver.find_element(By.CSS_SELECTOR, '#schEstandardDt_input').send_keys(Keys.DELETE)
driver.find_element(By.CSS_SELECTOR, '#schEstandardDt_input').send_keys(Keys.DELETE)
driver.find_element(By.CSS_SELECTOR, '#schEstandardDt_input').send_keys(Keys.BACKSPACE)
driver.find_element(By.CSS_SELECTOR, '#schEstandardDt_input').send_keys(Keys.BACKSPACE)
driver.find_element(By.CSS_SELECTOR, '#schEstandardDt_input').send_keys(Keys.BACKSPACE)
driver.find_element(By.CSS_SELECTOR, '#schEstandardDt_input').send_keys(Keys.BACKSPACE)
driver.find_element(By.CSS_SELECTOR, '#schEstandardDt_input').send_keys(end_date)

Select(driver.find_element(By.CSS_SELECTOR, '#selectBndTyp1_input_0')).select_by_index(1)
Select(driver.find_element(By.CSS_SELECTOR, '#selectBndTyp2_input_0')).select_by_index(1)
Select(driver.find_element(By.CSS_SELECTOR, '#selectBndTyp3_input_0')).select_by_index(1)
Select(driver.find_element(By.CSS_SELECTOR, '#selectBndTyp4_input_0')).select_by_index(1)
Select(driver.find_element(By.CSS_SELECTOR, '#selectbox_input_0')).select_by_index(2)

Select(driver.find_element(By.CSS_SELECTOR, '#selectTrm1_input_0')).select_by_index(5)
Select(driver.find_element(By.CSS_SELECTOR, '#selectTrm2_input_0')).select_by_index(6)
Select(driver.find_element(By.CSS_SELECTOR, '#selectTrm3_input_0')).select_by_index(7)
Select(driver.find_element(By.CSS_SELECTOR, '#selectTrm4_input_0')).select_by_index(8)

In [68]:
driver.execute_script('searchData()')

In [69]:
driver.execute_script('excelDownLoad()')

In [70]:
start_date = '20210101'
end_date = '20211231'

driver.find_element(By.CSS_SELECTOR, '#schSstandardDt_input').click()
driver.find_element(By.CSS_SELECTOR, '#schSstandardDt_input').send_keys(Keys.DELETE)
driver.find_element(By.CSS_SELECTOR, '#schSstandardDt_input').send_keys(Keys.DELETE)
driver.find_element(By.CSS_SELECTOR, '#schSstandardDt_input').send_keys(Keys.DELETE)
driver.find_element(By.CSS_SELECTOR, '#schSstandardDt_input').send_keys(Keys.DELETE)
driver.find_element(By.CSS_SELECTOR, '#schSstandardDt_input').send_keys(Keys.BACKSPACE)
driver.find_element(By.CSS_SELECTOR, '#schSstandardDt_input').send_keys(Keys.BACKSPACE)
driver.find_element(By.CSS_SELECTOR, '#schSstandardDt_input').send_keys(Keys.BACKSPACE)
driver.find_element(By.CSS_SELECTOR, '#schSstandardDt_input').send_keys(Keys.BACKSPACE)
driver.find_element(By.CSS_SELECTOR, '#schSstandardDt_input').send_keys(start_date)

driver.find_element(By.CSS_SELECTOR, '#schEstandardDt_input').click()
driver.find_element(By.CSS_SELECTOR, '#schEstandardDt_input').send_keys(Keys.DELETE)
driver.find_element(By.CSS_SELECTOR, '#schEstandardDt_input').send_keys(Keys.DELETE)
driver.find_element(By.CSS_SELECTOR, '#schEstandardDt_input').send_keys(Keys.DELETE)
driver.find_element(By.CSS_SELECTOR, '#schEstandardDt_input').send_keys(Keys.DELETE)
driver.find_element(By.CSS_SELECTOR, '#schEstandardDt_input').send_keys(Keys.BACKSPACE)
driver.find_element(By.CSS_SELECTOR, '#schEstandardDt_input').send_keys(Keys.BACKSPACE)
driver.find_element(By.CSS_SELECTOR, '#schEstandardDt_input').send_keys(Keys.BACKSPACE)
driver.find_element(By.CSS_SELECTOR, '#schEstandardDt_input').send_keys(Keys.BACKSPACE)
driver.find_element(By.CSS_SELECTOR, '#schEstandardDt_input').send_keys(end_date)

Select(driver.find_element(By.CSS_SELECTOR, '#selectBndTyp1_input_0')).select_by_index(1)
Select(driver.find_element(By.CSS_SELECTOR, '#selectBndTyp2_input_0')).select_by_index(1)
Select(driver.find_element(By.CSS_SELECTOR, '#selectBndTyp3_input_0')).select_by_index(1)
Select(driver.find_element(By.CSS_SELECTOR, '#selectBndTyp4_input_0')).select_by_index(1)
Select(driver.find_element(By.CSS_SELECTOR, '#selectbox_input_0')).select_by_index(2)

Select(driver.find_element(By.CSS_SELECTOR, '#selectTrm1_input_0')).select_by_index(9)
Select(driver.find_element(By.CSS_SELECTOR, '#selectTrm2_input_0')).select_by_index(10)
Select(driver.find_element(By.CSS_SELECTOR, '#selectTrm3_input_0')).select_by_index(11)
Select(driver.find_element(By.CSS_SELECTOR, '#selectTrm4_input_0')).select_by_index(12)

In [71]:
driver.execute_script('searchData()')

In [72]:
driver.execute_script('excelDownLoad()')

In [73]:
start_date = '20210101'
end_date = '20211231'

driver.find_element(By.CSS_SELECTOR, '#schSstandardDt_input').click()
driver.find_element(By.CSS_SELECTOR, '#schSstandardDt_input').send_keys(Keys.DELETE)
driver.find_element(By.CSS_SELECTOR, '#schSstandardDt_input').send_keys(Keys.DELETE)
driver.find_element(By.CSS_SELECTOR, '#schSstandardDt_input').send_keys(Keys.DELETE)
driver.find_element(By.CSS_SELECTOR, '#schSstandardDt_input').send_keys(Keys.DELETE)
driver.find_element(By.CSS_SELECTOR, '#schSstandardDt_input').send_keys(Keys.BACKSPACE)
driver.find_element(By.CSS_SELECTOR, '#schSstandardDt_input').send_keys(Keys.BACKSPACE)
driver.find_element(By.CSS_SELECTOR, '#schSstandardDt_input').send_keys(Keys.BACKSPACE)
driver.find_element(By.CSS_SELECTOR, '#schSstandardDt_input').send_keys(Keys.BACKSPACE)
driver.find_element(By.CSS_SELECTOR, '#schSstandardDt_input').send_keys(start_date)

driver.find_element(By.CSS_SELECTOR, '#schEstandardDt_input').click()
driver.find_element(By.CSS_SELECTOR, '#schEstandardDt_input').send_keys(Keys.DELETE)
driver.find_element(By.CSS_SELECTOR, '#schEstandardDt_input').send_keys(Keys.DELETE)
driver.find_element(By.CSS_SELECTOR, '#schEstandardDt_input').send_keys(Keys.DELETE)
driver.find_element(By.CSS_SELECTOR, '#schEstandardDt_input').send_keys(Keys.DELETE)
driver.find_element(By.CSS_SELECTOR, '#schEstandardDt_input').send_keys(Keys.BACKSPACE)
driver.find_element(By.CSS_SELECTOR, '#schEstandardDt_input').send_keys(Keys.BACKSPACE)
driver.find_element(By.CSS_SELECTOR, '#schEstandardDt_input').send_keys(Keys.BACKSPACE)
driver.find_element(By.CSS_SELECTOR, '#schEstandardDt_input').send_keys(Keys.BACKSPACE)
driver.find_element(By.CSS_SELECTOR, '#schEstandardDt_input').send_keys(end_date)

Select(driver.find_element(By.CSS_SELECTOR, '#selectBndTyp1_input_0')).select_by_index(1)
Select(driver.find_element(By.CSS_SELECTOR, '#selectBndTyp2_input_0')).select_by_index(1)
Select(driver.find_element(By.CSS_SELECTOR, '#selectBndTyp3_input_0')).select_by_index(1)
Select(driver.find_element(By.CSS_SELECTOR, '#selectBndTyp4_input_0')).select_by_index(1)
Select(driver.find_element(By.CSS_SELECTOR, '#selectbox_input_0')).select_by_index(2)

Select(driver.find_element(By.CSS_SELECTOR, '#selectTrm1_input_0')).select_by_index(13)
Select(driver.find_element(By.CSS_SELECTOR, '#selectTrm2_input_0')).select_by_index(14)
Select(driver.find_element(By.CSS_SELECTOR, '#selectTrm3_input_0')).select_by_index(15)
Select(driver.find_element(By.CSS_SELECTOR, '#selectTrm4_input_0')).select_by_index(16)

In [74]:
driver.execute_script('searchData()')

In [75]:
driver.execute_script('excelDownLoad()')

In [222]:
files = glob.glob(DOWNLOAD_PATH + '/기간별*.xls')
result = []
for file in files:
    df = pd.read_excel(file)
    index = df.iloc[2:, 0]
    values = df.iloc[2:, 1:].values.astype(float)
    columns = df.iloc[0, 1:].values
    df2 = pd.DataFrame(values, columns=columns, index=index)
    result.append(df2)
df3 = pd.concat(result, axis=1)/100
df3.columns = df3.columns.map(lambda x: TENOR_MAP.get(x, '#'))
df3 = df3.reset_index()
df3['일자'] = df3['일자'].str.replace('/', '')
df3 = df3.rename(columns={'일자': 'BASE_DATE'})
df3 = df3.melt(id_vars='BASE_DATE', var_name='MATURITY', value_name='INT_RATE')
df3['MATURITY'] = df3['MATURITY'].astype(float)
df3.insert(0, 'CURRENCY', 'KRW')
df3

,CURRENCY,BASE_DATE,MATURITY,INT_RATE
0,KRW,20211231,4.0,0.01925
1,KRW,20211230,4.0,0.01925
2,KRW,20211229,4.0,0.01890
3,KRW,20211228,4.0,0.01910
4,KRW,20211227,4.0,0.01897
...,...,...,...,...
3979,KRW,20210108,1.0,0.00633
3980,KRW,20210107,1.0,0.00636
3981,KRW,20210106,1.0,0.00636
3982,KRW,20210105,1.0,0.00636
